# HW8 (20')

<font size='4'>

For this assignment, it is a combination of jupyter notebook assignment and python scripts.

For Q1, please upload your outputs including codes and graphics to your own GitHub repository. <br> You will need to disclose your GitHub repository below.

For Q2, please submit this jupyter notebook as an HTML or PDF file.

First of all, print your name (First and Last) below.

## 0. Import relevant packages

In [9]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.formula.api as smf

## Q1. Convert your HW7 to python scripts. (10')

<font size='4'>

- Under your working directory, there should be a folder called `self_py_fun`.
- Create a new python file called `HW8Fun.py` and move previously defined functions `produce_trun_mean_cov()`, `plot_trunc_mean()`, and `plot_trunc_cov()` to that file. Make sure you import proper packages.
- Create another main file `HW8_main.py`.
- Import relevant packages, modules, and/or function.
- Copy the global variables and call your functions inside `HW8_main.py`.
- A major difference compared to HW7 is that you are asked to save those figures to your local working environment.
    - Create a new directory `K114` under your current working directory.
    - For mean functions, please save it as a `Mean.png` output using `plt.savefig()` function.
    - The changes should be made within `HW8Fun.py` rather than `HW8_main.py`.
    - For covariance matrices, please save them as `Covariance_Target.png`, `Covariance_Non-Target.png`, and `Covariance_All.png` outputs using the same function above.
    - To summarize, there should be **four** figures under `K114` folder.
- Upload your entire work to your GitHub repository via push button.

In [10]:
# Provide your GitHub repository link below in the Markdown chunk. Remember to make it public and make the link clickable.
# Do not include sensitive information in your GitHub repository.

## Q2. A real-world data anlaysis using `Pandas` and `Scipy` (10')

<font size='4'>

- Back to the `PTSD dataset.xlsx`, let's import the dataset and name it `ptsd_df`. (no point since everyone has done it a couple of times before.)

In [11]:
current_dir = os.getcwd()
ptsd_df = pd.read_excel('{}/data/PTSD dataset.xlsx'.format(current_dir), sheet_name='main_dataset')

### Q2.1. Univariate comparison (3')

<font size='4'> 
    
- Suppose that we would like to examine the utility/effect of an intervention program for patients with PTSD.
- We measure PCL5 scores at completion (`pcl5week_score.completion`) and PCL5 score at 3-month follow-up (`pcl5month_score.3_month_follow_up`). Let's assume the first score is pre-intervention and the second score is post-intervention.
- Report the summary statistics for each variable including mean, std, median, Q1, and Q3.
- Note that each patient will receive such two PCL5 scores. Use a appropriate statistical test to perform the univariate comparison. Report the outputing statistic and p-value.
- Before you run the statistic test, determine the data type and check the missingness of two columns. In particular, report the number of NA values for each variable.

In [12]:
print(ptsd_df.loc[:, ['pcl5week_score.completion', 'pcl5month_score.3_month_follow_up']].describe().T)
print()
print(ptsd_df.loc[:, 'pcl5week_score.completion'].isnull().sum())
print(ptsd_df.loc[:, 'pcl5month_score.3_month_follow_up'].isnull().sum())
print()
stat, pvalue = stats.ttest_rel(ptsd_df.loc[:, 'pcl5week_score.completion'], ptsd_df.loc[:,'pcl5month_score.3_month_follow_up'], nan_policy='omit')
print('stat={}'.format(stat))
print('p-value={}'.format(pvalue))

                                   count       mean        std  min   25%  \
pcl5week_score.completion          456.0  29.438596  19.373840  0.0  15.0   
pcl5month_score.3_month_follow_up  232.0  31.719828  18.812908  0.0  16.0   

                                    50%    75%   max  
pcl5week_score.completion          25.0  42.00  80.0  
pcl5month_score.3_month_follow_up  29.5  44.25  79.0  

27
251

stat=-4.709238006778945
p-value=4.370038747681667e-06


### Q2.2. Multiple Linear Regression (7')

<font size='4'>

- Select columns specified in the following code chunk and create a subset dataset named `ptsd_sub_df`.
- Fit a linear regression to examine the association between `caps_intake` (continuous outcome) and the remaining covariates (as predictors) using `ptsd_sub_df`.
    - Note that all covariates ending with `_code` are categorical variables.
- Use the instruction here to write the formula for linear regression in Python.
    - https://www.statsmodels.org/stable/example_formulas.html
- Report the output page including R2, adjusted R2, and parameter estimates, SE, 95% confidence intervals, and p-values.
- Provide a brief interpretation for all significant predictors (p<0.05) excluding the intercept.
- Relevant label information includes:
    - `employment_code`: 1: Employed, 2: Unemployed, 3: Retired, 4: Disabled/Unable to work, 5: Student, 6: Other.
    - `rank_code`: 1. Enlisted, 2: Officer, 3: Other

In [13]:
# The following column names are used for linear regression.
# Do not delete.
relevant_col_names = ['caps_intake', 'age_iop', 'gender_code', 'sexualorient_code', 'race_code', 'ethnicity_code', 
                      'education_code', 'employment_code',
                      'rank_code', 'branch_code', 'mdd_code', 'ctq_total_score', 'sexual_trauma', 'sud_code']

In [14]:
ptsd_sub_df = ptsd_df.loc[:, relevant_col_names]

lm_model = smf.ols('caps_intake ~ age_iop + C(gender_code) + C(sexualorient_code) + C(race_code) + C(ethnicity_code) ' + 
                   '+ C(education_code) + C(employment_code) + C(rank_code) + C(branch_code) + C(mdd_code) ' + 
                   '+ ctq_total_score + C(sexual_trauma) + C(sud_code)', data=ptsd_sub_df, missing='drop')

results = lm_model.fit()

In [15]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            caps_intake   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     1.796
Date:                Fri, 21 Nov 2025   Prob (F-statistic):            0.00697
Time:                        10:56:58   Log-Likelihood:                -1569.0
No. Observations:                 448   AIC:                             3200.
Df Residuals:                     417   BIC:                             3327.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [16]:
# Write your interpretations below:
